In [1]:
import pandas as pd
import bar_chart_race as bcr


######### IMPORTAÇÃO DO ARQUIVO #########


In [2]:
arquivo = 'C:\\Temp\\Futebol\\dados_serie_b.csv'

brasileirao = pd.read_csv(arquivo 
                          , sep=';'
                          , names=['NomTimeMandante'
                                   ,'PlacarMandante'
                                   ,'PlacarVisitante'
                                   ,'NomTimeVisitante'
                                   ,'NumRodada']
                         ,dtype={'NomTimeMandante':'str'
                                   ,'PlacarMandante':'long'
                                   ,'PlacarVisitante':'long'
                                   ,'NomTimeVisitante':'str'
                                   ,'NumRodada':'long'})

brasileirao['VTM'] = 0
brasileirao['EPT'] = 0
brasileirao['VTV'] = 0



######### DEFINIÇÃO DE VITORIA MANDANTE, VISITANTE OU EMPATE #########


In [3]:
brasileirao.loc[brasileirao['PlacarMandante'] > brasileirao['PlacarVisitante'], 'VTM'] = 1
brasileirao.loc[brasileirao['PlacarMandante'] == brasileirao['PlacarVisitante'], 'EPT'] = 1
brasileirao.loc[brasileirao['PlacarMandante'] < brasileirao['PlacarVisitante'], 'VTV'] = 1


######### DEFINIÇÃO DE NÚMERO DE VITORIAS E EMPATES #########


In [4]:
dfEmpate1 = brasileirao[brasileirao['EPT']==1][['NomTimeMandante','NumRodada','EPT']]
dfEmpate1['NumEmpate'] = 1
dfEmpate1['NumVitoria'] = 0
dfEmpate1.rename(columns={'NomTimeMandante': 'NomTime'},inplace=True)

dfEmpate2 = brasileirao[brasileirao['EPT']==1][['NomTimeVisitante','NumRodada','EPT']]
dfEmpate2['NumEmpate'] = 1
dfEmpate2['NumVitoria'] = 0
dfEmpate2.rename(columns={'NomTimeVisitante': 'NomTime'},inplace=True)

dfVitoriaMandante = brasileirao[brasileirao['VTM']==1][['NomTimeMandante','NumRodada','VTM']]
dfVitoriaMandante['NumEmpate'] = 0
dfVitoriaMandante['NumVitoria'] = 1
dfVitoriaMandante.rename(columns={'NomTimeMandante': 'NomTime'},inplace=True)

dfVitoriaVisitante = brasileirao[brasileirao['VTV']==1][['NomTimeVisitante','NumRodada','VTV']]
dfVitoriaVisitante['NumEmpate'] = 0
dfVitoriaVisitante['NumVitoria'] = 1
dfVitoriaVisitante.rename(columns={'NomTimeVisitante': 'NomTime'},inplace=True)



######### EMPILHANDO O MANDANTE E O VISITANTE EM UMA ÚNICA COLUNA #########


In [5]:
df2 = brasileirao[['NomTimeVisitante','PlacarVisitante','PlacarMandante','NumRodada']]
df2.rename(columns={'NomTimeVisitante': 'NomTime','PlacarVisitante':'GP','PlacarMandante':'GC'},inplace=True)


df1 = brasileirao[['NomTimeMandante','PlacarMandante','PlacarVisitante','NumRodada']]
df1.rename(columns={'NomTimeMandante': 'NomTime','PlacarMandante':'GP','PlacarVisitante':'GC'},inplace=True)

dfGP_GC = pd.merge(df1
            , df2
            , how='outer')

dfGP_GC = dfGP_GC.groupby(['NomTime'],as_index=False).agg(GP=('GP','sum'),GC=('GC','sum'))

dfGP_GC['SG'] = dfGP_GC['GP'] - dfGP_GC['GC']

C:\Users\lacer\AppData\Local\Temp\ipykernel_21128\4024880458.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2.rename(columns={'NomTimeVisitante': 'NomTime','PlacarVisitante':'GP','PlacarMandante':'GC'},inplace=True)
C:\Users\lacer\AppData\Local\Temp\ipykernel_21128\4024880458.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1.rename(columns={'NomTimeMandante': 'NomTime','PlacarMandante':'GP','PlacarVisitante':'GC'},inplace=True)



######### DEFININDO A PONTUAÇÃO GANHA NA RODADA #########


In [6]:
dfVitoriaMandante['NumPontosRodada'] = dfVitoriaMandante['VTM'] * 3
dfVitoriaVisitante['NumPontosRodada'] = dfVitoriaVisitante['VTV'] * 3
dfEmpate1['NumPontosRodada'] = dfEmpate1['EPT'] 
dfEmpate2['NumPontosRodada'] = dfEmpate2['EPT'] 


######### EMPILHANDO OS PONTOS DA RODADA #########


In [7]:

dfTabelaCampeonato = pd.merge(
                        (pd.merge(
                        (pd.merge(dfEmpate1
                              , dfEmpate2
                                  , how = 'outer'))
                        ,dfVitoriaMandante
                        , how = 'outer'))
                        ,dfVitoriaVisitante
                        , how = 'outer')[['NomTime','NumRodada','NumVitoria','NumEmpate','NumPontosRodada']]


dfTabelaCampeonato['Chave'] = dfTabelaCampeonato['NomTime'] + '_' + dfTabelaCampeonato['NumRodada'].astype(str)



######### ETAPA DE CRIAÇÃO DE REGISTROS INEXISTENTE DE RODADA #########

######### COMO NÃO CONTABILIZO AS DERROTAS, FICAM ALGUNS GAPS #########


In [8]:

rodadaTotal = [1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38]

time = dfTabelaCampeonato['NomTime'].unique()

df = pd.DataFrame(columns=['NomTime', 'NumRodada'])

for listaTimes in time:
    
    for listaRodada in rodadaTotal:
        
        df = df.append({'NomTime': listaTimes, 'NumRodada': str(listaRodada)}, ignore_index=True)

df['Chave'] = df['NomTime'] + '_' + df['NumRodada']


C:\Users\lacer\AppData\Local\Temp\ipykernel_21128\4089682687.py:11: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append({'NomTime': listaTimes, 'NumRodada': str(listaRodada)}, ignore_index=True)
C:\Users\lacer\AppData\Local\Temp\ipykernel_21128\4089682687.py:11: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append({'NomTime': listaTimes, 'NumRodada': str(listaRodada)}, ignore_index=True)
C:\Users\lacer\AppData\Local\Temp\ipykernel_21128\4089682687.py:11: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append({'NomTime': listaTimes, 'NumRodada': str(listaRodada)}, ignore_index=True)
C:\Users\lacer\AppData\Local\Temp\ipykernel_21128\4089682687.py:11: FutureWarning: The frame.append method is deprecated 

C:\Users\lacer\AppData\Local\Temp\ipykernel_21128\4089682687.py:11: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append({'NomTime': listaTimes, 'NumRodada': str(listaRodada)}, ignore_index=True)
C:\Users\lacer\AppData\Local\Temp\ipykernel_21128\4089682687.py:11: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append({'NomTime': listaTimes, 'NumRodada': str(listaRodada)}, ignore_index=True)
C:\Users\lacer\AppData\Local\Temp\ipykernel_21128\4089682687.py:11: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append({'NomTime': listaTimes, 'NumRodada': str(listaRodada)}, ignore_index=True)
C:\Users\lacer\AppData\Local\Temp\ipykernel_21128\4089682687.py:11: FutureWarning: The frame.append method is deprecated 

C:\Users\lacer\AppData\Local\Temp\ipykernel_21128\4089682687.py:11: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append({'NomTime': listaTimes, 'NumRodada': str(listaRodada)}, ignore_index=True)
C:\Users\lacer\AppData\Local\Temp\ipykernel_21128\4089682687.py:11: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append({'NomTime': listaTimes, 'NumRodada': str(listaRodada)}, ignore_index=True)
C:\Users\lacer\AppData\Local\Temp\ipykernel_21128\4089682687.py:11: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append({'NomTime': listaTimes, 'NumRodada': str(listaRodada)}, ignore_index=True)
C:\Users\lacer\AppData\Local\Temp\ipykernel_21128\4089682687.py:11: FutureWarning: The frame.append method is deprecated 

C:\Users\lacer\AppData\Local\Temp\ipykernel_21128\4089682687.py:11: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append({'NomTime': listaTimes, 'NumRodada': str(listaRodada)}, ignore_index=True)
C:\Users\lacer\AppData\Local\Temp\ipykernel_21128\4089682687.py:11: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append({'NomTime': listaTimes, 'NumRodada': str(listaRodada)}, ignore_index=True)
C:\Users\lacer\AppData\Local\Temp\ipykernel_21128\4089682687.py:11: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append({'NomTime': listaTimes, 'NumRodada': str(listaRodada)}, ignore_index=True)
C:\Users\lacer\AppData\Local\Temp\ipykernel_21128\4089682687.py:11: FutureWarning: The frame.append method is deprecated 

C:\Users\lacer\AppData\Local\Temp\ipykernel_21128\4089682687.py:11: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append({'NomTime': listaTimes, 'NumRodada': str(listaRodada)}, ignore_index=True)
C:\Users\lacer\AppData\Local\Temp\ipykernel_21128\4089682687.py:11: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append({'NomTime': listaTimes, 'NumRodada': str(listaRodada)}, ignore_index=True)
C:\Users\lacer\AppData\Local\Temp\ipykernel_21128\4089682687.py:11: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append({'NomTime': listaTimes, 'NumRodada': str(listaRodada)}, ignore_index=True)
C:\Users\lacer\AppData\Local\Temp\ipykernel_21128\4089682687.py:11: FutureWarning: The frame.append method is deprecated 

...

C:\Users\lacer\AppData\Local\Temp\ipykernel_21128\4089682687.py:11: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append({'NomTime': listaTimes, 'NumRodada': str(listaRodada)}, ignore_index=True)
C:\Users\lacer\AppData\Local\Temp\ipykernel_21128\4089682687.py:11: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append({'NomTime': listaTimes, 'NumRodada': str(listaRodada)}, ignore_index=True)
C:\Users\lacer\AppData\Local\Temp\ipykernel_21128\4089682687.py:11: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append({'NomTime': listaTimes, 'NumRodada': str(listaRodada)}, ignore_index=True)
C:\Users\lacer\AppData\Local\Temp\ipykernel_21128\4089682687.py:11: FutureWarning: The frame.append method is deprecated 

C:\Users\lacer\AppData\Local\Temp\ipykernel_21128\4089682687.py:11: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append({'NomTime': listaTimes, 'NumRodada': str(listaRodada)}, ignore_index=True)
C:\Users\lacer\AppData\Local\Temp\ipykernel_21128\4089682687.py:11: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append({'NomTime': listaTimes, 'NumRodada': str(listaRodada)}, ignore_index=True)
C:\Users\lacer\AppData\Local\Temp\ipykernel_21128\4089682687.py:11: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append({'NomTime': listaTimes, 'NumRodada': str(listaRodada)}, ignore_index=True)
C:\Users\lacer\AppData\Local\Temp\ipykernel_21128\4089682687.py:11: FutureWarning: The frame.append method is deprecated 

C:\Users\lacer\AppData\Local\Temp\ipykernel_21128\4089682687.py:11: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append({'NomTime': listaTimes, 'NumRodada': str(listaRodada)}, ignore_index=True)
C:\Users\lacer\AppData\Local\Temp\ipykernel_21128\4089682687.py:11: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append({'NomTime': listaTimes, 'NumRodada': str(listaRodada)}, ignore_index=True)
C:\Users\lacer\AppData\Local\Temp\ipykernel_21128\4089682687.py:11: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append({'NomTime': listaTimes, 'NumRodada': str(listaRodada)}, ignore_index=True)
C:\Users\lacer\AppData\Local\Temp\ipykernel_21128\4089682687.py:11: FutureWarning: The frame.append method is deprecated 

C:\Users\lacer\AppData\Local\Temp\ipykernel_21128\4089682687.py:11: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append({'NomTime': listaTimes, 'NumRodada': str(listaRodada)}, ignore_index=True)
C:\Users\lacer\AppData\Local\Temp\ipykernel_21128\4089682687.py:11: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append({'NomTime': listaTimes, 'NumRodada': str(listaRodada)}, ignore_index=True)
C:\Users\lacer\AppData\Local\Temp\ipykernel_21128\4089682687.py:11: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append({'NomTime': listaTimes, 'NumRodada': str(listaRodada)}, ignore_index=True)
C:\Users\lacer\AppData\Local\Temp\ipykernel_21128\4089682687.py:11: FutureWarning: The frame.append method is deprecated 

C:\Users\lacer\AppData\Local\Temp\ipykernel_21128\4089682687.py:11: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append({'NomTime': listaTimes, 'NumRodada': str(listaRodada)}, ignore_index=True)
C:\Users\lacer\AppData\Local\Temp\ipykernel_21128\4089682687.py:11: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append({'NomTime': listaTimes, 'NumRodada': str(listaRodada)}, ignore_index=True)
C:\Users\lacer\AppData\Local\Temp\ipykernel_21128\4089682687.py:11: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append({'NomTime': listaTimes, 'NumRodada': str(listaRodada)}, ignore_index=True)
C:\Users\lacer\AppData\Local\Temp\ipykernel_21128\4089682687.py:11: FutureWarning: The frame.append method is deprecated 


######### IGUALANDO O NUMERO DE RODADAS PARA TODOS OS TIMES #########


In [9]:
dfTabelaCampeonato = pd.merge(df
                , dfTabelaCampeonato
                , left_on='Chave'
                , right_on='Chave'
                , how = 'left')[['NomTime_x'
                                 ,'NumRodada_x'
                                 ,'NumVitoria'
                                 ,'NumPontosRodada'
                                 ,'NumEmpate']]

dfTabelaCampeonato.rename(columns={'NomTime_x': 'NomTime'
                                   ,'NumRodada_x':'NumRodada'},inplace=True)

dfTabelaCampeonato.fillna(0,inplace=True)

dfTabelaCampeonato['NumVitoria'] = dfTabelaCampeonato['NumVitoria'].astype(int)
dfTabelaCampeonato['NumPontosRodada'] = dfTabelaCampeonato['NumPontosRodada'].astype(int)
dfTabelaCampeonato['NumEmpate'] = dfTabelaCampeonato['NumEmpate'].astype(int)



######### ETAPA DE MONTAGEM DOS PONTOS GANHOS POR RODADA #########


In [10]:
dfCampBras = pd.DataFrame()
dfCampBrasLinha = pd.DataFrame()
dfLinha = pd.DataFrame()

dfTabelaCampeonato['NumRodada'] = dfTabelaCampeonato['NumRodada'].astype(int)

dfTabelaCampeonato.sort_values(by=['NumRodada','NomTime'],inplace=True)

dfTabelaCampeonato['Posicao'] = dfTabelaCampeonato.sort_values(by=['NomTime'
                                                                   ,'NumRodada'],ascending=True).groupby(['NomTime']).cumcount() + 1

dfTabelaCampeonato.sort_values(by=['NomTime'
                                   ,'Posicao'],ascending=True, inplace = True)

for listaNomTime in dfTabelaCampeonato['NomTime'].unique():
    
    for listaPosicao in dfTabelaCampeonato[dfTabelaCampeonato['NomTime']==listaNomTime]['Posicao']:
    
        dfLinha = dfTabelaCampeonato[(dfTabelaCampeonato['NomTime']==listaNomTime)
                                    & (dfTabelaCampeonato['Posicao']==listaPosicao)][['NomTime'
                                                                                      ,'NumRodada'
                                                                                      ,'NumPontosRodada'
                                                                                      ,'Posicao']]

        if listaPosicao == 1:
            
            dfLinha['PontoRodadaAnterior'] = 0

            dfLinha['TotalPontoRodada'] = dfLinha['NumPontosRodada'].astype(int)
        
            dfCampBras = pd.concat([dfCampBras, dfLinha])
            
            dfCampBras['TotalPontoRodada'].fillna(0, inplace=True)                        
            
        else:

            dfCampBrasLinha = dfCampBras[(dfCampBras['NomTime']==listaNomTime) 
                                         & (dfCampBras['Posicao']==listaPosicao)][['NomTime'
                                                                                   ,'NumRodada'
                                                                                   ,'NumPontosRodada'
                                                                                   ,'PontoRodadaAnterior'
                                                                                   ,'Posicao']]
            
            dfCampBrasLinha2 = dfTabelaCampeonato[(dfTabelaCampeonato['NomTime']==listaNomTime)
                                    & (dfTabelaCampeonato['Posicao']==listaPosicao)][['NomTime'
                                                                                      ,'NumRodada'
                                                                                      ,'NumPontosRodada'
                                                                                      ,'Posicao']]
            
            dfLinha['PontoRodadaAnterior'] = int(dfCampBras[(dfCampBras['NomTime']==listaNomTime) 
                                                            & (dfCampBras['Posicao']==listaPosicao-1)]['TotalPontoRodada'])
            
            dfLinha['NumPontosRodada'] = dfCampBrasLinha2['NumPontosRodada']
            dfLinha['NumRodada'] = dfCampBrasLinha2['NumRodada']
            dfLinha['Posicao'] = dfCampBrasLinha2['Posicao']
            dfLinha['TotalPontoRodada'] = int(dfLinha['PontoRodadaAnterior']) + int(dfLinha['NumPontosRodada'])
            
            dfCampBras = pd.concat([dfCampBras, dfLinha])

            
dfCampBras.sort_values(by=['NumRodada','NomTime'],inplace=True)



######### INVERSÃO DO DATAFRAME PRA MONTAGEM DO GRÁFICO #########

############## TRANSFORMAÇÃO DE LINHAS EM COLUNAS ###############


In [11]:

graficoCampBras =  pd.DataFrame(dfCampBras.groupby(['NomTime'
                                                      ,'NumRodada']).sum()['TotalPontoRodada']).unstack().T.droplevel(level=0)



######### GERAÇÃO DO GRÁFICO DE CORRIDA DE BARRAS #########


In [12]:
bcr.bar_chart_race( df =graficoCampBras ,
                    sort='desc',
                    n_bars=20,
                   orientation='h',
                    fixed_order=False,
                    #fixed_max=True,
                    steps_per_period=10,
                    #interpolate_period=False,
                    #label_bars=True,
                    bar_size=.95,
                    #period_length=500,
                    #figsize=(5, 5),
                    #title_size='',
                    bar_label_size=5,
                    tick_label_size=6,
                    shared_fontdict={'family' : 'Helvetica', 'color' : '.1'},
                    #scale='linear',
                    #writer=None,
                    #fig=None,
                    #cmap='dark12',
                    bar_kwargs={'alpha': .4},
                    #filter_column_colors=False,
                    title='Campeonato Brasileiro Serie B - 2022',
                    filename = 'C:\\Temp\\Futebol\\campeonato_brasileiro_serie_b.mp4'                
)

findfont: Font family ['Helvetica'] not found. Falling back to DejaVu Sans.
findfont: Font family ['Helvetica'] not found. Falling back to DejaVu Sans.
findfont: Font family ['Helvetica'] not found. Falling back to DejaVu Sans.
C:\Users\lacer\AppData\Roaming\Python\Python39\site-packages\bar_chart_race\_make_chart.py:286: UserWarning: FixedFormatter should only be used together with FixedLocator
  ax.set_yticklabels(self.df_values.columns)
C:\Users\lacer\AppData\Roaming\Python\Python39\site-packages\bar_chart_race\_make_chart.py:287: UserWarning: FixedFormatter should only be used together with FixedLocator
  ax.set_xticklabels([max_val] * len(ax.get_xticks()))
findfont: Font family ['Helvetica'] not found. Falling back to DejaVu Sans.



######### MONTAGEM DA TABELA FINAL DO CAMPEONATO #########


In [13]:
TabelaCampeonatoBrasileiro = dfTabelaCampeonato.groupby(['NomTime'],as_index=False).agg(Pontos=('NumPontosRodada','sum')
                                                          ,Vitorias=('NumVitoria','sum')
                                                          ,Empates=('NumEmpate','sum')
                                                          ).sort_values(by='Pontos',ascending=False)


TabelaCampeonatoBrasileiro['Jogos'] = dfTabelaCampeonato['NumRodada'].max()

TabelaCampeonatoBrasileiro['Derrotas'] = TabelaCampeonatoBrasileiro['Jogos'] - (
                                                        TabelaCampeonatoBrasileiro['Vitorias'] 
                                                        + TabelaCampeonatoBrasileiro['Empates']
                                                        )

TabelaCampeonatoBrasileiro['Posicao'] = TabelaCampeonatoBrasileiro.sort_values(by='Pontos',ascending=False).groupby(['Jogos']).cumcount() + 1

TabelaCampeonatoBrasileiro.sort_values(by=['Pontos'],ascending=False, inplace = True)

TabelaCampeonatoBrasileiro = pd.merge(left=TabelaCampeonatoBrasileiro
                                            , right=dfGP_GC
                                            , left_on='NomTime'
                                            , right_on='NomTime'
                                            )[['Posicao'
                                               ,'NomTime'
                                               ,'Jogos'
                                               ,'Pontos'
                                               ,'Vitorias'
                                               ,'Empates'
                                               ,'Derrotas'
                                               ,'GP'
                                               ,'GC'
                                               ,'SG']]

TabelaCampeonatoBrasileiro['%'] = round(((TabelaCampeonatoBrasileiro['Pontos'] / (TabelaCampeonatoBrasileiro['Jogos'] * 3)) * 100),2)

TabelaCampeonatoBrasileiro.loc[(TabelaCampeonatoBrasileiro['Posicao'] >= 1) & (TabelaCampeonatoBrasileiro['Posicao'] <= 4), 'Situacao'] = 'Classificado Série A'
TabelaCampeonatoBrasileiro.loc[(TabelaCampeonatoBrasileiro['Posicao'] >= 5) & (TabelaCampeonatoBrasileiro['Posicao'] <= 16), 'Situacao'] = 'Série B - 2023'
TabelaCampeonatoBrasileiro.loc[(TabelaCampeonatoBrasileiro['Posicao'] >= 17) & (TabelaCampeonatoBrasileiro['Posicao'] <= 20), 'Situacao'] = 'Rebaixado Série C'

TabelaCampeonatoBrasileiro


,Posicao,NomTime,Jogos,Pontos,Vitorias,Empates,Derrotas,GP,GC,SG,%,Situacao
0,1,Cruzeiro,38,78,23,9,6,57,26,31,68.42,Classificado Série A
1,2,Grêmio,38,65,17,14,7,50,26,24,57.02,Classificado Série A
2,3,Bahia,38,62,17,11,10,43,29,14,54.39,Classificado Série A
3,4,Vasco,38,62,17,11,10,48,36,12,54.39,Classificado Série A
4,5,Sampaio Corrêa,38,58,16,10,12,48,42,6,50.88,Série B - 2023
5,6,Ituano,38,57,15,12,11,42,34,8,50.00,Série B - 2023
6,7,Sport,38,57,15,12,11,37,31,6,50.00,Série B - 2023
7,8,Criciúma,38,56,14,14,10,43,31,12,49.12,Série B - 2023
8,9,Londrina,38,53,14,11,13,36,37,-1,46.49,Série B - 2023
9,10,Guarani,38,51,13,12,13,33,36,-3,44.74,Série B - 2023
